<a href="https://colab.research.google.com/github/xjdr-alt/entropix/blob/main/entropix.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" /></a>

<b><center><font size=14em>Stateful Metaflow Service & UI</font></center></b>
<center><em><font size=12em>(2/2) Consume</font></em></center>

<em>The herein notebook is the continuance of the <a href="metaflow_service.ipynb" target="_blank">(1/2) start</a> Google Colab notebook</em>

<hr />

In [1]:
from google.colab import drive
# grant all permission or it fails
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
tunnel_url = None

In [3]:
# below, must be the same Google Drive location
# as the one set in the "Metaflow Service" notebook
MF_ROOT = "/content/drive/MyDrive/Metaflow_hf"

In [4]:
!pip install metaflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.2 MB/s eta 0:00:00


In [5]:
%%writefile hello_world_flow.py
from metaflow import FlowSpec, step, current, card
from metaflow.cards import Markdown

# Define the HelloWorldFlow using Metaflow's FlowSpec
class HelloWorldFlow(FlowSpec):

    @step
    def start(self):
        print("Hello, World!")
        self.next(self.flow_card)

    @card(id='default')
    @step
    def flow_card(self):
        print("blabla")
        current.card['default'].append(Markdown("blabla"))
        self.next(self.end)

    @step
    def end(self):
        print("Flow Finished")

if __name__ == '__main__':
    HelloWorldFlow()

Writing hello_world_flow.py


In [13]:
datastore_dir = f"{MF_ROOT}/local_datastore/"

# when prompted, enter the URL
# of the tunnel which you established
# in the "Metaflow Service" Colab notebook
import requests
if tunnel_url is not None:
    try:
        response = requests.get(f"{tunnel_url}/service/ping")
        if response.status_code != 200 or response.text != "pong":
            tunnel_url = input("Enter an active tunnel URL:\n")
        else:
            print(tunnel_url)
    except:
        tunnel_url = input("The former endpoint is not reachable. " +
                           "Enter an active tunnel URL:\n")
else:
    tunnel_url = input("Enter an active tunnel URL:\n")

# Launch flow run
for i in range(2):
    ! echo "Running iteration {i}" && \
      export METAFLOW_SERVICE_URL={tunnel_url}/service/ && \
      export METAFLOW_DEFAULT_METADATA=service && \
      export METAFLOW_DEFAULT_DATASTORE='local' && \
      export METAFLOW_DATASTORE_SYSROOT_LOCAL={datastore_dir} && \
      export USERNAME=user && \
      cd {datastore_dir} && \
      python /content/hello_world_flow.py run

https://lbs-staffing-driver-subsequent.trycloudflare.com
Running iteration 0
Metaflow 2.12.25 executing HelloWorldFlow for user:user
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2024-10-08 19:02:17.733 Workflow starting (run-id 509):
2024-10-08 19:02:20.041 [509/start/2034 (pid 114582)] Task is starting.
2024-10-08 19:02:20.989 [509/start/2034 (pid 114582)] Hello, World!
2024-10-08 19:02:21.894 [509/start/2034 (pid 114582)] Task finished successfully.
2024-10-08 19:02:22.955 [509/flow_card/2035 (pid 114597)] Task is starting.
2024-10-08 19:02:24.020 [509/flow_card/2035 (pid 114597)] blabla
2024-10-08 19:02:30.183 [509/flow_card/2035 (pid 114597)] Task finished successfully.
2024-10-08 19:02:31.034 [509/end/2036 (pid 114636)] Task is starting.
2024-10-08 19:02:32.265 [509/end/2036 (pid 114636)] Flow Finished
2024-10-08 19:02:32.996 [509/end/2036 (pid 114636)] Task finished successfully.
2024-10-08 19:02:33.053 Do